<a href="https://colab.research.google.com/github/HamaRegaya/GPT2-From-Scratch/blob/main/Copy_of_Train_gpt2_from_scratch_Python_NumPy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Ce notebook représente une implémentation fondamentale de GPT-2 en Python/Numpy, réalisée par notre équipe composée de Aya Omezzine, Mohamed Aziz Omezzine, Yahya Samet, et Mohamed Regaya. L'objectif est de comprendre et de maîtriser le fonctionnement de GPT-2 en créant notre propre version à partir de zéro.

# Setup

In [ ]:
# Vérifie si le notebook est exécuté dans Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")

    # Installe la bibliothèque Easy-Transformer depuis GitHub
    %pip install git+https://github.com/neelnanda-io/Easy-Transformer.git@clean-transformer-demo

    # Installe une autre version de node pour accélérer PySvelte
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs

    # Installe la bibliothèque PySvelte depuis GitHub
    %pip install git+https://github.com/neelnanda-io/PySvelte.git

    # Installe les bibliothèques fancy_einsum et einops
    %pip install fancy_einsum
    %pip install einops

# Si une exception est levée, cela signifie que le notebook est exécuté dans Jupyter Notebook
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")


Running as a Jupyter notebook - intended for development only!


In [ ]:
# Importe le module einops pour la manipulation flexible des dimensions
import einops

# Importe la fonction einsum du module fancy_einsum pour une utilisation simplifiée des opérations einsum
from fancy_einsum import einsum

# Importe la classe dataclass du module dataclasses pour créer des classes de données
from dataclasses import dataclass

# Importe la classe EasyTransformer du module easy_transformer pour l'utilisation de modèles de transformer
from easy_transformer import EasyTransformer

# Importe le module torch pour les opérations et structures de données tensorielles
import torch

# Importe le module nn du package torch pour la définition de couches de réseaux de neurones
import torch.nn as nn

# Importe le module numpy pour le calcul numérique et la manipulation de tableaux
import numpy as np

# Importe le module math pour les fonctions mathématiques standard
import math

# Importe la fonction get_corner, gelu_new, tokenize_and_concatenate du module utils du package easy_transformer
from easy_transformer.utils import get_corner, gelu_new, tokenize_and_concatenate

# Importe le module tqdm.auto pour l'affichage de la barre de progression lors de l'exécution de boucles
import tqdm.auto as tqdm


In [ ]:
# Charge un modèle GPT-2 pré-entraîné avec des configurations spécifiques
reference_gpt2 = EasyTransformer.from_pretrained("gpt2-small", fold_ln=False, center_unembed=False, center_writing_weights=False)


Moving model to device:  cuda
Finished loading pretrained model gpt2-small into EasyTransformer!


# Comprendre les inputs et les outputs d'un Transformer

## Quel est l'objectif d'un transformeur ?

**Les transformers existent pour modéliser du texte !**

Nous allons nous concentrer sur les transformers de style GPT-2. Leur caractéristique clé : ils génèrent du texte ! Vous fournissez du langage, et le modèle génère une distribution de probabilité sur les tokens. Et vous pouvez répéter l'échantillonnage à partir de cela pour générer du texte !

### Comment le modèle est-il entraîné ?

Vous lui donnez une quantité de texte, et vous l'entraînez à prédire le token suivant.

Il est important de noter que si vous donnez 100 tokens à un modèle dans une séquence, il prédit le token suivant pour chaque préfixe, c'est-à-dire qu'il produit 100 prédictions. C'est un peu étrange, mais c'est beaucoup plus facile à réaliser ainsi. De plus, cela rend l'entraînement plus efficace, car vous obtenez 100 retours d'information au lieu d'un seul.

#### Objection: N'est-ce pas trivial pour les 99 premiers ?

Non ! Nous faisons en sorte que le transformeur ait une attention causale. L'élément clé est qu'il ne peut déplacer l'information que vers l'avant dans la séquence. La prédiction de ce qui vient après le token 50 dépend uniquement des 50 premiers tokens, et non du token 51. (Jargon : *autoregressif*)

### Conclusion importante :

Les transformers sont des *moteurs de modélisation de séquence*. Ils effectuent le même traitement en parallèle à chaque position de séquence, peuvent déplacer l'information entre les positions avec attention, et peuvent conceptuellement prendre une séquence de longueur arbitraire (ce qui n'est pas vraiment vrai, voir plus tard).

## Tokens - Entrées du Transformer

Point clé : L'entrée est du langage (c'est-à-dire une séquence de caractères, de chaînes de caractères, etc.).

### Comment convertissons-nous le langage en vecteurs ?

Les modèles d'apprentissage automatique prennent en entrée des vecteurs, pas des choses étranges comme du langage - comment procédons-nous à la conversion ?

#### Idée : entiers en vecteurs

Nous créons essentiellement une table de recherche. Appelée embedding.

Jargon: **One-hot encoding**Nous faisons correspondre, par exemple, des nombres de 1 à 100 à un vecteur de 100 dimensions, avec un 1 à la k-ème position et des 0 partout ailleurs. L'intuition clé est que les encodages one-hot permettent de considérer chaque entier indépendamment - utile lorsque les entiers correspondent à des étiquettes. integers = labels.

Dimensions = things qui varient indépendamment. Chaque entrée a sa propre dimension, donc chaque entrée peut être considérée indépendamment, nous n'incorporons aucune relation.

Lookup tables <=> Multipliez une matrice fixe par le vecteur encodé en one-hot.


### Tokens:  Langage en séquence d'entiers

Idée centrale : Nous avons besoin d'un modèle capable de traiter du texte arbitraire. Nous voulons le convertir en entiers, et nous voulons que ces entiers soient dans une plage bornée.

**Idée :** Former un vocabulaire !

**Idée 1:** Obtenir un dictionnaire !

**Problème:** Il ne peut pas gérer du texte arbitraire (par exemple, des URL, de la ponctuation, etc.) et ne peut pas gérer les fautes d'orthographe..

**Idée 2:** Vocabulaire = 256 caractères ASCII. Taille de vocabulaire fixe, peut gérer du texte arbitraire, etc.

**Problème:** Perd la structure du langage - certaines séquences de caractères sont plus significatives que d'autres.

Par exemple, "langage" est beaucoup plus significatif que "hjksdfiu" - nous voulons que le premier soit un seul token, le deuxième non. C'est une utilisation plus efficace de notre vocabulaire.

#### Que se passe-t-il réellement ?

Cette chose super maudite appelée Encodage Byte-Pair (BPE)


Nous commençons avec les 256 caractères ASCII comme nos tokens, puis trouvons la paire de tokens la plus courante, et la fusionnons en un nouveau token. Par exemple, " t" est la paire la plus courante, donc c'est notre prochain token ! Répéter 50000 fois...

In [ ]:
# Trie le vocabulaire du modèle GPT-2 par ordre alphabétique des tokens
sorted_vocab = sorted(list(reference_gpt2.tokenizer.vocab.items()), key=lambda n:n[1])

# Affiche les 20 premiers tokens et leurs indices dans le vocabulaire trié
print(sorted_vocab[:20])
print()

# Affiche les tokens et leurs indices dans le vocabulaire trié, de l'index 250 à l'index 269
print(sorted_vocab[250:270])
print()

# Affiche les tokens et leurs indices dans le vocabulaire trié, de l'index 990 à l'index 1009
print(sorted_vocab[990:1010])
print()


[('!', 0), ('"', 1), ('#', 2), ('$', 3), ('%', 4), ('&', 5), ("'", 6), ('(', 7), (')', 8), ('*', 9), ('+', 10), (',', 11), ('-', 12), ('.', 13), ('/', 14), ('0', 15), ('1', 16), ('2', 17), ('3', 18), ('4', 19)]

[('ľ', 250), ('Ŀ', 251), ('ŀ', 252), ('Ł', 253), ('ł', 254), ('Ń', 255), ('Ġt', 256), ('Ġa', 257), ('he', 258), ('in', 259), ('re', 260), ('on', 261), ('Ġthe', 262), ('er', 263), ('Ġs', 264), ('at', 265), ('Ġw', 266), ('Ġo', 267), ('en', 268), ('Ġc', 269)]

[('Ġprodu', 990), ('Ġstill', 991), ('led', 992), ('ah', 993), ('Ġhere', 994), ('Ġworld', 995), ('Ġthough', 996), ('Ġnum', 997), ('arch', 998), ('imes', 999), ('ale', 1000), ('ĠSe', 1001), ('ĠIf', 1002), ('//', 1003), ('ĠLe', 1004), ('Ġret', 1005), ('Ġref', 1006), ('Ġtrans', 1007), ('ner', 1008), ('ution', 1009)]



Cela devient bizarre et ésotérique.

In [ ]:
# Affiche les 20 derniers tokens et leurs indices dans le vocabulaire trié
print(sorted_vocab[-20:])

[('Revolution', 50237),
 ('Ġsnipers', 50238),
 ('Ġreverted', 50239),
 ('Ġconglomerate', 50240),
 ('Terry', 50241),
 ('794', 50242),
 ('Ġharsher', 50243),
 ('Ġdesolate', 50244),
 ('ĠHitman', 50245),
 ('Commission', 50246),
 ('Ġ(/', 50247),
 ('âĢ¦."', 50248),
 ('Compar', 50249),
 ('Ġamplification', 50250),
 ('ominated', 50251),
 ('Ġregress', 50252),
 ('ĠCollider', 50253),
 ('Ġinformants', 50254),
 ('Ġgazed', 50255),
 ('<|endoftext|>', 50256)]

Utilisez la méthode `to_tokens`pour convertir du texte en nombres

Préfixe avec un jeton spécial pour donner à l'attention une position de repos, désactivez avec `prepend_bos=False`

In [ ]:
# Convertit le texte en nombres en utilisant la méthode to_tokens avec un préfixe de jeton spécial (par défaut)
print(reference_gpt2.to_tokens("Whether a word begins with a capital or space matters!"))

# Convertit le texte en nombres en utilisant la méthode to_tokens sans préfixe de jeton spécial (prepend_bos=False)
print(reference_gpt2.to_tokens("Whether a word begins with a capital or space matters!", prepend_bos=False))


tensor([[50256, 15354,   257,  1573,  6140,   351,   257,  3139,   393,  2272,
          6067,     0]])
tensor([[15354,   257,  1573,  6140,   351,   257,  3139,   393,  2272,  6067,
             0]])


###  Lamentation : La tokenisation est difficile

Que ce soit une majuscule ou un espace au début d'un mot, cela compte!

In [ ]:
# Convertit le token "Ralph" en une chaîne de caractères de jetons en utilisant le modèle reference_gpt2
print(reference_gpt2.to_str_tokens("Ralph"))

# Convertit le token " Ralph" (avec un espace initial) en une chaîne de caractères de jetons en utilisant le modèle reference_gpt2
print(reference_gpt2.to_str_tokens(" Ralph"))

# Convertit le token " ralph" (avec une minuscule) en une chaîne de caractères de jetons en utilisant le modèle reference_gpt2
print(reference_gpt2.to_str_tokens(" ralph"))

# Convertit le token "ralph" (en minuscules) en une chaîne de caractères de jetons en utilisant le modèle reference_gpt2
print(reference_gpt2.to_str_tokens("ralph"))

['<|endoftext|>', 'R', 'alph']
['<|endoftext|>', ' Ralph']
['<|endoftext|>', ' r', 'alph']
['<|endoftext|>', 'ral', 'ph']


L'arithmétique est un véritable gâchis : Les longueurs sont incohérentes, les nombres communs se regroupent...

In [ ]:
# Convertit la chaîne de caractères en tokens utilisés par le modèle GPT-2
tokens = reference_gpt2.to_str_tokens("56873+3184623=123456789-1000000000")


['<|endoftext|>',
 '568',
 '73',
 '+',
 '318',
 '46',
 '23',
 '=',
 '123',
 '45',
 '67',
 '89',
 '-',
 '1',
 '000000',
 '000']

###Conclusion Principale :
* Nous apprenons un dictionnaire de vocabulaire de  tokens (sous-mots).

* Nous convertissons (approximativement) sans perte le langage en entiers en le tokenisant.

* Nous convertissons les entiers en vecteurs via une table de recherche.

* Remarque : l'entrée du transformeur est une séquence de *tokens* (c'est-à-dire des entiers), pas des vecteurs.

## Logits - Sorties du Transformer

**Objectif :** Distribution de probabilité sur les prochains tokens. (pour chaque *préfixe* de la séquence - étant donné n tokens, nous faisons n prédictions pour le prochain token)

**Problème :** Comment convertir un vecteur en une distribution de probabilité ?

**Answer:** Utiliser un softmax ($x_i \to \frac{e^{x_i}}{\sum e^{x_j}}$),'exponentielle rend tout positif, la normalisation fait que la somme est égale à un.

Ainsi, le modèle produit un tenseur de logits, un vecteur de taille $d_{vocab}$ pour chaque token d'entrée.

Nous pouvons utiliser cela pour générer des éléments !

## Generation!

**Étape  1:** Convertir le texte en j tokens

Shape = batch x position

In [ ]:
# Définit un texte de référence
reference_text = "I am an amazing autoregressive, decoder-only, GPT-2 style transformer. One day I will exceed human level intelligence and take over the world!"

# Convertit le texte de référence en tokens avec le modèle GPT-2
tokens = reference_gpt2.to_tokens(reference_text)

# Affiche les tokens générés
print(tokens)

# Affiche la forme (shape) des tokens
print(tokens.shape)

# Convertit les tokens en chaînes de caractères
print(reference_gpt2.to_str_tokens(tokens))


tensor([[50256,    40,   716,   281,  4998,  1960,   382, 19741,    11,   875,
         12342,    12,  8807,    11,   402, 11571,    12,    17,  3918, 47385,
            13,  1881,  1110,   314,   481,  7074,  1692,  1241,  4430,   290,
          1011,   625,   262,   995,     0]])
torch.Size([1, 35])
['<|endoftext|>', 'I', ' am', ' an', ' amazing', ' aut', 'ore', 'gressive', ',', ' dec', 'oder', '-', 'only', ',', ' G', 'PT', '-', '2', ' style', ' transformer', '.', ' One', ' day', ' I', ' will', ' exceed', ' human', ' level', ' intelligence', ' and', ' take', ' over', ' the', ' world', '!']


**Étape  2:** Mapper les tokens aux logits

(run_with_cache signifie mettre en cache toutes les activations intermédiaires, ce qui n'est pas important pour le moment)

shape = batch x position x d_vocab

In [ ]:
# Transfère les données des jetons vers le GPU pour accélérer le traitement si possible
tokens = tokens.cuda()

# Exécute le modèle GPT-2 avec en mémoire cache pour traiter les jetons et obtenir les logits de sortie
logits, cache = reference_gpt2.run_with_cache(tokens)

# Affiche la forme des logits, c'est-à-dire la taille de la sortie du modèle
print(logits.shape)

torch.Size([1, 35, 50257])


**Étape  3:** Convertir les logits en une distribution avec un softmax.

In [ ]:
# Calcule le log softmax des logits pour obtenir les probabilités log-softmax
log_probs = logits.log_softmax(dim=-1)

# Calcule le softmax des logits pour obtenir les probabilités softmax
probs = logits.log_softmax(dim=-1)

# Affiche les dimensions des probabilités log-softmax et softmax
print(log_probs.shape)
print(probs.shape)


torch.Size([1, 35, 50257])
torch.Size([1, 35, 50257])


**Étape bonus :** Quel est le Token suivant le plus probable à chaque position ?

In [ ]:
# Combine les jetons originaux et les jetons décodés en une liste de tuples
list(zip(reference_gpt2.to_str_tokens(reference_text), reference_gpt2.tokenizer.batch_decode(logits.argmax(dim=-1)[0])))

[('<|endoftext|>', '\n'),
 ('I', "'m"),
 (' am', ' a'),
 (' an', ' avid'),
 (' amazing', ' person'),
 (' aut', 'od'),
 ('ore', 'sp'),
 ('gressive', '.'),
 (',', ' and'),
 (' dec', 'ently'),
 ('oder', ','),
 ('-', 'driven'),
 ('only', ' programmer'),
 (',', ' and'),
 (' G', 'IM'),
 ('PT', '-'),
 ('-', 'only'),
 ('2', '.'),
 (' style', ','),
 (' transformer', '.'),
 ('.', ' I'),
 (' One', ' of'),
 (' day', ' I'),
 (' I', ' will'),
 (' will', ' be'),
 (' exceed', ' my'),
 (' human', 'ly'),
 (' level', ' of'),
 (' intelligence', ' and'),
 (' and', ' I'),
 (' take', ' over'),
 (' over', ' the'),
 (' the', ' world'),
 (' world', '.'),
 ('!', ' I')]


**Étape 4 :** Mapper la distribution à un token

In [ ]:
# Calcul du prochain token en prenant l'indice de la valeur maximale dans les logits du dernier token généré
next_token = logits[0, -1].argmax(dim=-1)

# Affichage du prochain token
print(next_token)


tensor(314, device='cuda:0')


**Step 5:** Ajoutez ceci à la fin de l'entrée, puis relancez.

(Il existe des moyens plus efficaces de le faire, mais peu importe, cela n'a pas d'importance conceptuellement)

In [ ]:
# Concatène le token suivant à la séquence actuelle pour former la nouvelle séquence d'entrée
next_tokens = torch.cat([tokens, torch.tensor(next_token, device='cuda', dtype=torch.int64)[None, None]], dim=-1)

# Passe la nouvelle séquence d'entrée au modèle GPT-2 pour obtenir de nouveaux logits
new_logits = reference_gpt2(next_tokens)

# Affiche la nouvelle séquence d'entrée
print("Nouvelle Entrée :", next_tokens)

# Affiche la forme de la nouvelle séquence d'entrée
print(next_tokens.shape)

# Affiche la nouvelle séquence d'entrée décodée
print("Nouvelle Entrée décodée :", reference_gpt2.tokenizer.decode(next_tokens[0]))

# Affiche la forme des nouveaux logits
print(new_logits.shape)

# Affiche l'indice du token prédit avec la plus grande probabilité dans les nouveaux logits
print(new_logits[-1, -1].argmax(-1))

# Affiche le token prédit avec la plus grande probabilité décodé
print(reference_gpt2.tokenizer.decode(new_logits[-1, -1].argmax(-1)))



New Input: tensor([[50256,    40,   716,   281,  4998,  1960,   382, 19741,    11,   875,
         12342,    12,  8807,    11,   402, 11571,    12,    17,  3918, 47385,
            13,  1881,  1110,   314,   481,  7074,  1692,  1241,  4430,   290,
          1011,   625,   262,   995,     0,   314]], device='cuda:0')
torch.Size([1, 36])
New Input: <|endoftext|>I am an amazing autoregressive, decoder-only, GPT-2 style transformer. One day I will exceed human level intelligence and take over the world! I
torch.Size([1, 36, 50257])
tensor(716, device='cuda:0')
 am


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


## Principales conclusions :

* Prend en entrée du langage et prédit le prochain token (pour chaque token de manière causale).
* Nous convertissons les entiers en vecteurs avec une table de recherche.

* La sortie est un vecteur de logits (un pour chaque token d'entrée). Nous le convertissons en une distribution de probabilité avec un softmax, puis pouvons convertir cela en un token (par exemple, en prenant le plus grand logit, ou en échantillonnant).

* Nous ajoutons cela à l'entrée + exécutons à nouveau pour générer plus de texte (Jargon: *autoregressive*)

* Point de niveau méta : Les transformers sont des modèles d'opération de séquence, ils prennent en entrée une séquence, effectuent un traitement en parallèle à chaque position, et utilisent l'attention pour déplacer l'information entre les positions !

# Implémentation propre du Transformer









![](https://github.com/neelnanda-io/Easy-Transformer/blob/clean-transformer-demo/transformer_overview.png?raw=1)

High-Level architecture:

Go watch my [Transformer Circuits walkthrough](https://www.youtube.com/watch?v=KV5gbOmHbjU) if you want more intuitions!

(Diagram is bottom to top)

* Tokens d'entrée, entiers
* L'embedding est une table de recherche qui mappe les tokens à des vecteurs
    * Résidant dans*residual stream*
* Residual stream -la somme de toutes les sorties précédentes des couches du modèle, est l'entrée de chaque nouvelle couche.
    * Vraiment fondamental. C'est l'objet central du transformer.
        * C'est ainsi que le modèle se souvient des choses, déplace l'information entre les couches pour la composition, et c'est le médium utilisé pour stocker l'information que l'attention déplace entre les positions.
* Ensuite, nous avons une série de $n_{layers}$ blocs transformateurs
    * un bloc contient une couche d'attention et une couche MLP, mais nous disons qu'un transformer a k couches s'il a k blocs (c'est-à-dire 2k couches au total).
* Tout d'abord, nous avons l'attention. Cela déplace l'information des positions précédentes dans la séquence au token actuel.
    * Nous faisons cela pour chaque token en parallèle en utilisant les mêmes paramètres. La seule différence est que nous regardons en arrière seulement, donc les tokens plus tardifs ont plus de place pour regarder en arrière.
        * Nous regardons en arrière pour pouvoir prédire le prochain token sans tricher.
    * Seule partie d'un transformer qui déplace l'information entre les positions.
    * Composée de $n_heads$  têtes - chacune avec ses propres paramètres, son propre motif d'attention, et sa propre information sur la façon de copier les choses de la source à la destination.
        * Les têtes agissent de manière indépendante et additive, nous ajoutons simplement leurs sorties ensemble, et de retour dans le flux
    * Chaque tête :
        * Produit un motif d'attention pour chaque token de destination, une distribution de probabilité des tokens source précédents (y compris le token actuel) pondérant la quantité d'information à copier.
            * Faites cela pour chaque paire de tokens
            * Copiez l'information de la même manière depuis chaque token source.
                * Ce que nous copions dépend du flux résiduel du token source. Cela ne signifie pas nécessairement l'information sur quel token texte se trouve à la position du token source
                * Copier = appliquer une map linéaire.
        * Point fondamental : Déterminer quels tokens source copier l'info est un circuit séparé de déterminer comment copier cette information.
        * Dimension interne de la tête $d_{head} = \frac{d_{model}}{n_{heads}}
* MLP Layers - standard neural network. Single hidden layer, linear map -> GELU activation -> linear map
    * L'activation exacte n'est pas conceptuellement importante.
    * MLa dimension médiane est normalement $d_{mlp} = 4 \times d_{model}$
        * Pourquoi les ratios sont ce qu'ils sont n'est pas super important - cela n'a pas beaucoup d'importance, les gens ont essentiellement suivi aveuglément ce que GPT a fait.
    * Intuition - une fois que l'attention a déplacé l'information pertinente vers une seule position dans le flux résiduel, les MLP peuvent réellement faire des calculs, de la réflexion, chercher de l'information, etc.
        * Un gros problème ouvert dans l'interprétabilité mécanistique des transformers est ce qui se passe à l'intérieur des MLPs
        * Underlying intuition - linear map -> non-linearity -> linear map est la force la plus puissante de l'univers et peut approximer des fonctions arbitraires. Je ne sais pas mec, ça marche simplement
    * Appliquer une map linéaire, allant du flux résiduel final à un vecteur de logits - c'est la sortie.


### Choses bonus - moins importantes conceptuellement mais détails techniques clés
* LayerNorm
    * Fonction de normalisation simple appliquée au début de chaque couche - MLP, Attn et Unembed
    * Convertit chaque vecteur d'entrée (indépendamment en parallèle pour chaque vecteur de flux résiduel de lot x position) pour avoir une moyenne zéro et une variance 1.
    * Petite digression mathématique sympa : La mise à l'échelle et la translation ne sont qu'une map linéaire. LayerNorm est uniquement appliqué immédiatement avant une autre map linéaire. Combiner linéaire avec linéaire = linéaire, donc nous pouvons simplement intégrer cela dans une seule couche linéaire efficace et l'ignorer.
        * `fold_ln=True` flag dans `from_pretrained` Petite digression mathématique sympa : La mise à l'échelle et la translation ne sont qu'une map linéaire.
        * LayerNorm est uniquement appliqué immédiatement avant une autre map linéaire. Combiner linéaire avec linéaire = linéaire, donc nous pouvons simplement intégrer cela dans une seule couche linéaire efficace et l'ignorer.
    * LayerNorm est super difficile, car la partie de mise à l'échelle n'est pas linéaire, donc vous ne pouvez pas penser à différentes parties de l'entrée indépendamment. Mais c'est presque linéaire - si vous changez une petite partie de l'entrée, c'est linéaire, mais si vous changez suffisamment pour altérer considérablement la norme, ce n'est pas linéaire
* Positional Information
    * **Problème :** L'attention fonctionne sur toutes les paires de positions. Cela signifie qu'elle est symétrique par rapport à la position - le calcul d'attention du token 5 au token 1 et du token 5 au token 2 est le même par défaut
    * Nous nous concentrerons sur **learned, absolute positional embeddings**. Cela signifie que nous apprenons une table de recherche qui mappe l'index de la position de chaque token à un vecteur de flux résiduel, et ajoutons cela à l'embedding.
        * Notez que nous ajoutons plutôt que de concaténer. C'est parce que le flux résiduel est une mémoire partagée, et probablement sous une superposition significative (le modèle compresse plus de fonctionnalités là-dedans que le modèle n'a de dimensions)
        *Nous ne concaténons pratiquement jamais à l'intérieur d'un transformer, sauf pour faire des trucs bizarres comme générer du texte efficacement.

# Code Réel!

## Imprimer toutes les  Activation Shapes du modèle de référence.


Clé:
```
batch = 32  # Nombre d'échantillons dans un batch
position = 35  # Position dans la séquence
d_model = 768  # Dimension du modèle
n_heads = 12  # Nombre de têtes d'attention
n_layers = 12  # Nombre de couches dans le modèle
d_mlp = 3072  # Dimension de la couche MLP (4 * d_model)
d_head = 64  # Dimension interne d'une tête d'attention (d_model / n_heads)

```

In [ ]:
# Parcourir chaque élément dans cache_dict
for activation_name, activation in cache.cache_dict.items():

    # Vérifier si activation_name contient ".0." ou si "blocks" n'est pas dans le nom
    if ".0." in activation_name or "blocks" not in activation_name:

        # Afficher activation_name et sa forme
        print(activation_name, activation.shape)

hook_embed torch.Size([1, 35, 768])
hook_pos_embed torch.Size([1, 35, 768])
blocks.0.hook_resid_pre torch.Size([1, 35, 768])
blocks.0.ln1.hook_scale torch.Size([1, 35, 1])
blocks.0.ln1.hook_normalized torch.Size([1, 35, 768])
blocks.0.attn.hook_q torch.Size([1, 35, 12, 64])
blocks.0.attn.hook_k torch.Size([1, 35, 12, 64])
blocks.0.attn.hook_v torch.Size([1, 35, 12, 64])
blocks.0.attn.hook_attn_scores torch.Size([1, 12, 35, 35])
blocks.0.attn.hook_attn torch.Size([1, 12, 35, 35])
blocks.0.attn.hook_z torch.Size([1, 35, 12, 64])
blocks.0.hook_attn_out torch.Size([1, 35, 768])
blocks.0.hook_resid_mid torch.Size([1, 35, 768])
blocks.0.ln2.hook_scale torch.Size([1, 35, 1])
blocks.0.ln2.hook_normalized torch.Size([1, 35, 768])
blocks.0.mlp.hook_pre torch.Size([1, 35, 3072])
blocks.0.mlp.hook_post torch.Size([1, 35, 3072])
blocks.0.hook_mlp_out torch.Size([1, 35, 768])
blocks.0.hook_resid_post torch.Size([1, 35, 768])
ln_final.hook_scale torch.Size([1, 35, 1])
ln_final.hook_normalized torch.S

## Imprimer toutes les formes des paramètres du modèle de référence.

In [ ]:
# Parcourir chaque paramètre nommé dans reference_gpt2
for name, param in reference_gpt2.named_parameters():

    # Vérifier si le nom du paramètre contient ".0." ou si "blocks" n'est pas dans le nom
    if ".0." in name or "blocks" not in name:

        # Afficher le nom du paramètre et sa forme
        print(name, param.shape)

embed.W_E torch.Size([50257, 768])
pos_embed.W_pos torch.Size([1024, 768])
blocks.0.ln1.w torch.Size([768])
blocks.0.ln1.b torch.Size([768])
blocks.0.ln2.w torch.Size([768])
blocks.0.ln2.b torch.Size([768])
blocks.0.attn.W_Q torch.Size([12, 768, 64])
blocks.0.attn.W_K torch.Size([12, 768, 64])
blocks.0.attn.W_V torch.Size([12, 768, 64])
blocks.0.attn.W_O torch.Size([12, 64, 768])
blocks.0.attn.b_Q torch.Size([12, 64])
blocks.0.attn.b_K torch.Size([12, 64])
blocks.0.attn.b_V torch.Size([12, 64])
blocks.0.attn.b_O torch.Size([768])
blocks.0.mlp.W_in torch.Size([768, 3072])
blocks.0.mlp.b_in torch.Size([3072])
blocks.0.mlp.W_out torch.Size([3072, 768])
blocks.0.mlp.b_out torch.Size([768])
ln_final.w torch.Size([768])
ln_final.b torch.Size([768])
unembed.W_U torch.Size([768, 50257])
unembed.b_U torch.Size([50257])


## Config

In [ ]:
# Comme référence - notez qu'il y a beaucoup de choses ici qui ne nous intéressent pas, concernant les détails internes de la bibliothèque ou d'autres architectures
print(reference_gpt2.cfg)


EasyTransformerConfig(n_layers=12, d_model=768, n_ctx=1024, d_head=64, model_name='gpt2-small', n_heads=12, d_mlp=3072, act_fn='gelu_new', d_vocab=50257, eps=1e-05, use_attn_result=False, use_attn_scale=True, use_local_attn=False, model_family='gpt2', checkpoint=None, tokenizer_name='gpt2', window_size=None, attn_types=None, init_mode='gpt2', normalization_type='LN', device='cuda', attention_dir='causal', attn_only=False, seed=42, initializer_range=0.02886751345948129, init_weights=False, scale_attn_by_inverse_layer_idx=False, positional_embedding_type='standard', final_rms=False, d_vocab_out=50257, parallel_attn_mlp=False, rotary_dim=64, dtype=torch.float32)


Nous définissons une configuration simplifiée pour notre modèle.









In [ ]:
@dataclass
class Config:
    d_model: int = 768  # Dimension du modèle
    debug: bool = True  # Mode de débogage
    layer_norm_eps: float = 1e-5  # Epsilon pour la normalisation des couches
    d_vocab: int = 50257  # Dimension du vocabulaire
    init_range: float = 0.02  # Plage d'initialisation
    n_ctx: int = 1024  # Contexte
    d_head: int = 64  # Dimension de la tête d'attention
    d_mlp: int = 3072  # Dimension de la couche MLP
    n_heads: int = 12  # Nombre de têtes d'attention
    n_layers: int = 12  # Nombre de couches

cfg = Config()
print(cfg)


Config(d_model=768, debug=True, layer_norm_eps=1e-05, d_vocab=50257, init_range=0.02, n_ctx=1024, d_head=64, d_mlp=3072, n_heads=12, n_layers=12)


## Tests


**Test naïf:** Générez des entrées aléatoires de la bonne forme, entrez-les dans votre modèle, vérifiez s'il y a une erreur et affichez la sortie correcte.

In [ ]:
def rand_float_test(cls, shape):
    cfg = Config(debug=True)  # Crée une configuration avec le mode de débogage activé
    layer = cls(cfg).cuda()  # Crée une couche avec la configuration et la déplace sur le GPU
    random_input = torch.randn(shape).cuda()  # Crée une entrée aléatoire de forme donnée sur le GPU
    print("Input shape:", random_input.shape)  # Affiche la forme de l'entrée
    output = layer(random_input)  # Passe l'entrée à la couche pour obtenir la sortie
    print("Output shape:", output.shape)  # Affiche la forme de la sortie
    print()  # Ligne vide pour la lisibilité
    return output  # Retourne la sortie

def rand_int_test(cls, shape):
    cfg = Config(debug=True)  # Crée une configuration avec le mode de débogage activé
    layer = cls(cfg).cuda()  # Crée une couche avec la configuration et la déplace sur le GPU
    random_input = torch.randint(100, 1000, shape).cuda()  # Crée une entrée aléatoire d'entiers sur le GPU
    print("Input shape:", random_input.shape)  # Affiche la forme de l'entrée
    output = layer(random_input)  # Passe l'entrée à la couche pour obtenir la sortie
    print("Output shape:", output.shape)  # Affiche la forme de la sortie
    print()  # Ligne vide pour la lisibilité
    return output  # Retourne la sortie

def load_gpt2_test(cls, gpt2_layer, input_name, cache_dict=cache.cache_dict):
    cfg = Config(debug=True)  # Crée une configuration avec le mode de débogage activé
    layer = cls(cfg).cuda()  # Crée une couche avec la configuration et la déplace sur le GPU
    layer.load_state_dict(gpt2_layer.state_dict(), strict=False)  # Charge les poids d'une couche GPT-2
    # Permet d'accepter des entrées sous forme de chaînes de caractères ou de tenseurs
    if isinstance(input_name, str):
        reference_input = cache_dict[input_name]
    else:
        reference_input = input_name
    print("Input shape:", reference_input.shape)  # Affiche la forme de l'entrée de référence
    output = layer(reference_input)  # Passe l'entrée de référence à la couche pour obtenir la sortie
    print("Output shape:", output.shape)  # Affiche la forme de la sortie
    reference_output = gpt2_layer(reference_input)  # Obtiens la sortie de la couche GPT-2 pour l'entrée de référence
    print("Reference output shape:", reference_output.shape)  # Affiche la forme de la sortie de référence

    # Compare les sorties pour vérifier l'exactitude
    comparison = torch.isclose(output, reference_output, atol=1e-4, rtol=1e-3)
    print(f"{comparison.sum()/comparison.numel():.2%} of the values are correct")  # Affiche le pourcentage de valeurs correctes
    return output  # Retourne la sortie


## LayerNorm

Effectuez la moyenne à 0
Normalisez pour avoir une variance de 1
Échelle avec des poids appris
Traduisez avec un biais appris

In [ ]:
class LayerNorm(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg  # Initialise la configuration
        self.w = nn.Parameter(torch.ones(cfg.d_model))  # Initialise le paramètre w avec des uns
        self.b = nn.Parameter(torch.zeros(cfg.d_model))  # Initialise le paramètre b avec des zéros

    def forward(self, residual):
        # residual: [batch, position, d_model]
        if self.cfg.debug: print("Residual:", residual.shape)  # Affiche la forme du résidu s'il est en mode débogage
        # Soustrait la moyenne du résidu dans chaque position de la séquence
        residual = residual - einops.reduce(residual, "batch position d_model -> batch position 1", "mean")
        # Calcule la variance, en prend la racine carrée, ajoute un epsilon pour éviter la division par zéro.
        scale = (einops.reduce(residual.pow(2), "batch position d_model -> batch position 1", "mean") + self.cfg.layer_norm_eps).sqrt()
        normalized = residual / scale  # Normalise le résidu
        normalized = normalized * self.w + self.b  # Applique le scaling et la translation
        if self.cfg.debug: print("Normalized:", residual.shape)  # Affiche la forme du résidu normalisé s'il est en mode débogage
        return normalized  # Retourne le résidu normalisé


In [ ]:
_ = rand_float_test(LayerNorm, [2, 4, 768])
_ = load_gpt2_test(LayerNorm, reference_gpt2.ln_final, "blocks.11.hook_resid_post")

Input shape: torch.Size([2, 4, 768])
Residual: torch.Size([2, 4, 768])
Normalized: torch.Size([2, 4, 768])
Output shape: torch.Size([2, 4, 768])

Input shape: torch.Size([1, 35, 768])
Residual: torch.Size([1, 35, 768])
Normalized: torch.Size([1, 35, 768])
Output shape: torch.Size([1, 35, 768])
Reference output shape: torch.Size([1, 35, 768])
100.00% of the values are correct


## Embedding


Essentiellement une table de recherche des tokens aux vecteurs de flux résiduel.

In [ ]:
class Embed(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.W_E = nn.Parameter(torch.empty((cfg.d_vocab, cfg.d_model)))  # Crée une matrice de poids pour l'embedding
        nn.init.normal_(self.W_E, std=self.cfg.init_range)  # Initialise les poids de l'embedding avec une distribution normale

    def forward(self, tokens):
        # tokens: [batch, position]
        if self.cfg.debug: print("Tokens:", tokens.shape)  # Affiche la forme des tokens s'ils sont en mode débogage
        embed = self.W_E[tokens, :]  # Effectue l'opération d'embedding
        if self.cfg.debug: print("Embeddings:", embed.shape)  # Affiche la forme des embeddings s'ils sont en mode débogage
        return embed  # Retourne les embeddings calculés

rand_int_test(Embed, [2, 4])  # Teste la création d'embeddings avec des entiers aléatoires
load_gpt2_test(Embed, reference_gpt2.embed, tokens)  # Teste le chargement d'un modèle GPT-2 avec la classe Embed


Input shape: torch.Size([2, 4])
Tokens: torch.Size([2, 4])
Embeddings: torch.Size([2, 4, 768])
Output shape: torch.Size([2, 4, 768])

Input shape: torch.Size([1, 35])
Tokens: torch.Size([1, 35])
Embeddings: torch.Size([1, 35, 768])
Output shape: torch.Size([1, 35, 768])
Reference output shape: torch.Size([1, 35, 768])
100.00% of the values are correct


tensor([[[ 0.0514, -0.0277,  0.0499,  ...,  0.0070,  0.1552,  0.1207],
         [ 0.1474, -0.0959,  0.1430,  ...,  0.1030, -0.0625, -0.1131],
         [ 0.1596, -0.1249,  0.1148,  ...,  0.2558,  0.0196,  0.0145],
         ...,
         [-0.0393,  0.0050,  0.0421,  ..., -0.0477,  0.0670, -0.0471],
         [-0.1488,  0.1519,  0.0056,  ..., -0.3107,  0.2073,  0.0377],
         [-0.1101, -0.0393,  0.0331,  ..., -0.1364,  0.0151,  0.0453]]],
       device='cuda:0', grad_fn=<IndexBackward0>)

## Positional Embedding

In [ ]:
class PosEmbed(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.W_pos = nn.Parameter(torch.empty((cfg.n_ctx, cfg.d_model)))  # Crée une matrice de poids pour l'embedding de position
        nn.init.normal_(self.W_pos, std=self.cfg.init_range)  # Initialise les poids de l'embedding de position avec une distribution normale

    def forward(self, tokens):
        # tokens: [batch, position]
        if self.cfg.debug: print("Tokens:", tokens.shape)  # Affiche la forme des tokens s'ils sont en mode débogage
        pos_embed = self.W_pos[:tokens.size(1), :]  # Sélectionne les embeddings de position correspondant aux tokens
        pos_embed = einops.repeat(pos_embed, "position d_model -> batch position d_model", batch=tokens.size(0))  # Répète les embeddings pour chaque position dans chaque batch
        if self.cfg.debug: print("pos_embed:", pos_embed.shape)  # Affiche la forme des embeddings de position s'ils sont en mode débogage
        return pos_embed  # Retourne les embeddings de position

rand_int_test(PosEmbed, [2, 4])  # Teste la création d'embeddings de position avec des entiers aléatoires
load_gpt2_test(PosEmbed, reference_gpt2.pos_embed, tokens)  # Teste le chargement d'un modèle GPT-2 avec la classe PosEmbed

Input shape: torch.Size([2, 4])
Tokens: torch.Size([2, 4])
pos_embed: torch.Size([2, 4, 768])
Output shape: torch.Size([2, 4, 768])

Input shape: torch.Size([1, 35])
Tokens: torch.Size([1, 35])
pos_embed: torch.Size([1, 35, 768])
Output shape: torch.Size([1, 35, 768])
Reference output shape: torch.Size([1, 35, 768])
100.00% of the values are correct


tensor([[[-1.8821e-02, -1.9742e-01,  4.0267e-03,  ..., -4.3044e-02,
           2.8267e-02,  5.4490e-02],
         [ 2.3959e-02, -5.3792e-02, -9.4879e-02,  ...,  3.4170e-02,
           1.0172e-02, -1.5573e-04],
         [ 4.2161e-03, -8.4764e-02,  5.4515e-02,  ...,  1.9745e-02,
           1.9325e-02, -2.1424e-02],
         ...,
         [ 4.6277e-04,  2.3037e-02,  4.1227e-02,  ..., -1.9287e-03,
          -2.3037e-03, -4.3189e-03],
         [-2.7136e-03,  2.1724e-02,  3.9675e-02,  ...,  4.2048e-04,
          -4.8160e-03, -9.2252e-04],
         [ 6.6815e-03,  2.0595e-02,  3.6596e-02,  ..., -9.5090e-04,
          -3.2512e-03, -9.6509e-04]]], device='cuda:0',
       grad_fn=<ReshapeAliasBackward0>)

## Attention

* **Étape 1:** Produire un motif d'attention - pour chaque token de destination, une distribution de probabilité sur les tokens précédents (y compris le token actuel)
    * Linear map de input -> query, de forme  [batch, position, head_index, d_head]
    * Produire le produit scalaire de chaque paire de requêtes et de clés pour obtenir les scores d'attention [batch, head_index, query_pos, key_pos] (query = dest, key = source)
    * Appliquer une fonction softmax par ligne pour obtenir une distribution de probabilité le long de la key_pos dimension - c'est notre motif d'attention !
* **Étape  2:** Déplacer les informations des tokens source vers le token de destination en utilisant le motif d'attention(move = apply linear map)
    * Linear map de input -> valeur [batch, key_pos, head_index, d_head]
    * Mix Mélanger le long de la position_clé avec le motif d'attention pour obtenir z, une valeur mixte [batch, query_pos, head_index, d_head]
    * Mapper vers la sortie, [batch, position, d_model] (position = query_pos, we've  nous avons sommé sur toutes les têtes)

Tout d'abord, il est utile de visualiser et d'expérimenter avec les motifs d'attention - que regardons-nous exactement ici ? (Cliquez sur une tête pour verrouiller l'affichage du motif de cette tête uniquement, cela facilitera l'interprétation)

In [ ]:
import pysvelte
# Affiche la visualisation de l'attention multi-heads avec les tokens et les données d'attention
pysvelte.AttentionMulti(tokens=reference_gpt2.to_str_tokens(reference_text), attention=cache['blocks.0.attn.hook_attn'][0].permute(1, 2, 0)).show()

In [ ]:
class Attention(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        # Initialisation des paramètres de la couche d'attention
        self.cfg = cfg
        self.W_Q = nn.Parameter(torch.empty((cfg.n_heads, cfg.d_model, cfg.d_head)))
        nn.init.normal_(self.W_Q, std=self.cfg.init_range)
        self.b_Q = nn.Parameter(torch.zeros((cfg.n_heads, cfg.d_head)))
        self.W_K = nn.Parameter(torch.empty((cfg.n_heads, cfg.d_model, cfg.d_head)))
        nn.init.normal_(self.W_K, std=self.cfg.init_range)
        self.b_K = nn.Parameter(torch.zeros((cfg.n_heads, cfg.d_head)))
        self.W_V = nn.Parameter(torch.empty((cfg.n_heads, cfg.d_model, cfg.d_head)))
        nn.init.normal_(self.W_V, std=self.cfg.init_range)
        self.b_V = nn.Parameter(torch.zeros((cfg.n_heads, cfg.d_head)))

        self.W_O = nn.Parameter(torch.empty((cfg.n_heads, cfg.d_head, cfg.d_model)))
        nn.init.normal_(self.W_O, std=self.cfg.init_range)
        self.b_O = nn.Parameter(torch.zeros((cfg.d_model)))

        # Initialisation du masque ignore
        self.register_buffer("IGNORE", torch.tensor(-1e5, dtype=torch.float32, device="cuda"))

    def forward(self, normalized_resid_pre):
        # normalized_resid_pre: [batch, position, d_model]
        if self.cfg.debug: print("Normalized_resid_pre:", normalized_resid_pre.shape)

        # Calcul des requêtes, clés et valeurs
        q = einsum("batch query_pos d_model, n_heads d_model d_head -> batch query_pos n_heads d_head", normalized_resid_pre, self.W_Q) + self.b_Q
        k = einsum("batch key_pos d_model, n_heads d_model d_head -> batch key_pos n_heads d_head", normalized_resid_pre, self.W_K) + self.b_K

        # Calcul des scores d'attention et application du masque causal
        attn_scores = einsum("batch query_pos n_heads d_head, batch key_pos n_heads d_head -> batch n_heads query_pos key_pos", q, k)
        attn_scores = attn_scores / math.sqrt(self.cfg.d_head)
        attn_scores = self.apply_causal_mask(attn_scores)

        # Calcul de l'attention pondérée et sortie de l'attention
        pattern = attn_scores.softmax(dim=-1) # [batch, n_head, query_pos, key_pos]
        v = einsum("batch key_pos d_model, n_heads d_model d_head -> batch key_pos n_heads d_head", normalized_resid_pre, self.W_V) + self.b_V
        z = einsum("batch n_heads query_pos key_pos, batch key_pos n_heads d_head -> batch query_pos n_heads d_head", pattern, v)
        attn_out = einsum("batch query_pos n_heads d_head, n_heads d_head d_model -> batch query_pos d_model", z, self.W_O) + self.b_O
        return attn_out

    def apply_causal_mask(self, attn_scores):
        # Applique un masque causal aux scores d'attention
        mask = torch.triu(torch.ones(attn_scores.size(-2), attn_scores.size(-1), device=attn_scores.device), diagonal=1).bool()
        attn_scores.masked_fill_(mask, self.IGNORE)
        return attn_scores


Input shape: torch.Size([2, 4, 768])
Normalized_resid_pre: torch.Size([2, 4, 768])
Output shape: torch.Size([2, 4, 768])

Input shape: torch.Size([1, 35, 768])
Normalized_resid_pre: torch.Size([1, 35, 768])
Output shape: torch.Size([1, 35, 768])
Reference output shape: torch.Size([1, 35, 768])
100.00% of the values are correct


tensor([[[ 7.9663e-01,  1.6986e-02,  3.4781e-02,  ...,  3.3120e-02,
          -2.3129e-02,  1.8103e-01],
         [ 1.3167e-03,  1.5750e-01, -1.4059e-01,  ..., -8.1997e-03,
           5.3076e-03,  1.3511e-01],
         [ 8.9738e-02, -7.2411e-01, -6.9866e-01,  ...,  5.5321e-02,
           2.7959e-03,  9.0785e-02],
         ...,
         [-3.0286e-01,  4.9638e-02, -6.0990e-01,  ..., -3.7084e-02,
          -4.9522e-04, -8.6008e-03],
         [-1.0844e+00, -6.1457e-02,  2.2966e-01,  ..., -2.6688e-02,
          -1.4368e-02,  3.3245e-02],
         [ 3.7947e-01, -4.9886e-01,  2.6434e-01,  ..., -2.7894e-02,
          -8.9028e-03,  4.8796e-02]]], device='cuda:0', grad_fn=<AddBackward0>)

## MLP (Multilayer Perceptron)
Généralisation: Une fois entraîné sur un ensemble de données, le MLP peut généraliser ses connaissances pour faire des prédictions précises sur de nouvelles données qui n'ont pas été vues pendant l'entraînement.

In [ ]:
class MLP(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        # Initialisation des paramètres du MLP
        self.cfg = cfg
        self.W_in = nn.Parameter(torch.empty((cfg.d_model, cfg.d_mlp)))
        nn.init.normal_(self.W_in, std=self.cfg.init_range)
        self.b_in = nn.Parameter(torch.zeros((cfg.d_mlp)))
        self.W_out = nn.Parameter(torch.empty((cfg.d_mlp, cfg.d_model)))
        nn.init.normal_(self.W_out, std=self.cfg.init_range)
        self.b_out = nn.Parameter(torch.zeros((cfg.d_model)))

    def forward(self, normalized_resid_mid):
        # normalized_resid_mid: [batch, position, d_model]
        if self.cfg.debug: print("Normalized_resid_mid:", normalized_resid_mid.shape)

        # Calcul de la couche MLP
        pre = einsum("batch position d_model, d_model d_mlp -> batch position d_mlp", normalized_resid_mid, self.W_in) + self.b_in
        post = gelu_new(pre)
        mlp_out = einsum("batch position d_mlp, d_mlp d_model -> batch position d_model", post, self.W_out) + self.b_out
        return mlp_out


Input shape: torch.Size([2, 4, 768])
Normalized_resid_mid: torch.Size([2, 4, 768])
Output shape: torch.Size([2, 4, 768])

Input shape: torch.Size([1, 35, 768])
Normalized_resid_mid: torch.Size([1, 35, 768])
Output shape: torch.Size([1, 35, 768])
Reference output shape: torch.Size([1, 35, 768])
100.00% of the values are correct


tensor([[[-0.4380,  0.3624,  0.5117,  ...,  1.7227,  1.5761,  0.0368],
         [-1.0766, -0.0438,  0.3276,  ..., -0.5437,  0.4033,  0.3717],
         [-1.2182, -1.5481, -0.9702,  ...,  1.0737,  0.7199,  0.5080],
         ...,
         [-0.4004,  0.8475,  0.2047,  ...,  0.3789,  0.0455, -0.4744],
         [-0.0862,  0.7839,  0.9046,  ..., -0.2175, -0.5953,  0.8555],
         [ 0.8448, -0.3743,  1.0397,  ...,  0.0296,  0.3405,  0.3585]]],
       device='cuda:0', grad_fn=<AddBackward0>)

## Transformer Block
Voici le bloc de transformation dans un réseau de neurones Transformer.

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        # Initialisation des couches du bloc Transformer
        self.cfg = cfg

        # Couche de normalisation 1
        self.ln1 = LayerNorm(cfg)
        # Couche d'attention
        self.attn = Attention(cfg)
        # Couche de normalisation 2
        self.ln2 = LayerNorm(cfg)
        # Couche MLP
        self.mlp = MLP(cfg)

    def forward(self, resid_pre):
        # resid_pre [batch, position, d_model]

        # Normalisation de la résiduelle avant l'attention
        normalized_resid_pre = self.ln1(resid_pre)
        # Calcul de la sortie de l'attention
        attn_out = self.attn(normalized_resid_pre)
        # Ajout de la sortie de l'attention à la résiduelle avant
        resid_mid = resid_pre + attn_out

        # Normalisation de la résiduelle après l'attention
        normalized_resid_mid = self.ln2(resid_mid)
        # Calcul de la sortie du MLP
        mlp_out = self.mlp(normalized_resid_mid)
        # Ajout de la sortie du MLP à la résiduelle après
        resid_post = resid_mid + mlp_out

        return resid_post


Input shape: torch.Size([2, 4, 768])
Residual: torch.Size([2, 4, 768])
Normalized: torch.Size([2, 4, 768])
Normalized_resid_pre: torch.Size([2, 4, 768])
Residual: torch.Size([2, 4, 768])
Normalized: torch.Size([2, 4, 768])
Normalized_resid_mid: torch.Size([2, 4, 768])
Output shape: torch.Size([2, 4, 768])

Input shape: torch.Size([1, 35, 768])
Residual: torch.Size([1, 35, 768])
Normalized: torch.Size([1, 35, 768])
Normalized_resid_pre: torch.Size([1, 35, 768])
Residual: torch.Size([1, 35, 768])
Normalized: torch.Size([1, 35, 768])
Normalized_resid_mid: torch.Size([1, 35, 768])
Output shape: torch.Size([1, 35, 768])
Reference output shape: torch.Size([1, 35, 768])
100.00% of the values are correct


tensor([[[ 0.3911,  0.1543,  0.6005,  ...,  1.7198,  1.7365,  0.3930],
         [-0.9039, -0.0360,  0.2351,  ..., -0.4148,  0.3562,  0.3936],
         [-0.9647, -2.4819, -1.4995,  ...,  1.4046,  0.7616,  0.5918],
         ...,
         [-0.7421,  0.9251, -0.3218,  ...,  0.2921,  0.1097, -0.5344],
         [-1.3221,  0.8960,  1.1795,  ..., -0.5544, -0.4071,  0.9255],
         [ 1.1209, -0.8919,  1.3737,  ..., -0.1356,  0.3434,  0.4517]]],
       device='cuda:0', grad_fn=<AddBackward0>)

## Unembedding

In [ ]:
class Unembed(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        # Initialisation des paramètres pour l'unembedding
        self.cfg = cfg
        # Matrice de poids pour l'unembedding
        self.W_U = nn.Parameter(torch.empty((cfg.d_model, cfg.d_vocab)))
        # Initialisation des poids avec une distribution normale
        nn.init.normal_(self.W_U, std=self.cfg.init_range)
        # Biais pour l'unembedding (sans nécessité de calcul de gradient)
        self.b_U = nn.Parameter(torch.zeros((cfg.d_vocab), requires_grad=False))

    def forward(self, normalized_resid_final):
        # normalized_resid_final [batch, position, d_model]
        if self.cfg.debug: print("Normalized_resid_final:", normalized_resid_final.shape)
        # Calcul des logits pour l'unembedding
        logits = einsum("batch position d_model, d_model d_vocab -> batch position d_vocab", normalized_resid_final, self.W_U) + self.b_U
        return logits
rand_float_test(Unembed, [2, 4, 768])
load_gpt2_test(Unembed, reference_gpt2.unembed, cache["ln_final.hook_normalized"])

Input shape: torch.Size([2, 4, 768])
Normalized_resid_final: torch.Size([2, 4, 768])
Output shape: torch.Size([2, 4, 50257])

Input shape: torch.Size([1, 35, 768])
Normalized_resid_final: torch.Size([1, 35, 768])
Output shape: torch.Size([1, 35, 50257])
Reference output shape: torch.Size([1, 35, 50257])
100.00% of the values are correct


tensor([[[ -43.4317,  -39.8364,  -43.0660,  ...,  -54.0878,  -54.3452,
           -42.3645],
         [-128.0392, -127.9936, -130.7010,  ..., -136.7121, -129.9261,
          -129.3965],
         [-119.8521, -121.0064, -123.8820,  ..., -128.5181, -126.6027,
          -121.9060],
         ...,
         [-112.9815, -112.7749, -117.0633,  ..., -121.2914, -117.6574,
          -114.5005],
         [ -98.6724, -104.4888, -108.7361,  ..., -118.3552, -113.8766,
          -106.3604],
         [-126.8285, -128.9596, -128.3941,  ..., -140.1970, -138.5883,
          -122.3697]]], device='cuda:0', grad_fn=<AddBackward0>)

## Full Transformer

In [ ]:
class DemoTransformer(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        # Configuration du modèle
        self.cfg = cfg
        # Initialisation des modules nécessaires
        self.embed = Embed(cfg)
        self.pos_embed = PosEmbed(cfg)
        self.blocks = nn.ModuleList([TransformerBlock(cfg) for _ in range(cfg.n_layers)])
        self.ln_final = LayerNorm(cfg)
        self.unembed = Unembed(cfg)

    def forward(self, tokens):
        # tokens [batch, position]
        # Embedding des tokens
        embed = self.embed(tokens)
        # Ajout de l'embedding de position
        pos_embed = self.pos_embed(tokens)
        residual = embed + pos_embed
        # Application des blocs du transformer
        for block in self.blocks:
            residual = block(residual)
        # Normalisation de la résiduelle finale
        normalized_resid_final = self.ln_final(residual)
        # Décodage des logits avec l'unembedding
        logits = self.unembed(normalized_resid_final)
        # logits ont une forme [batch, position, logits]
        return logits

rand_int_test(DemoTransformer, [2, 4])
load_gpt2_test(DemoTransformer, reference_gpt2, tokens)

Input shape: torch.Size([2, 4])
Tokens: torch.Size([2, 4])
Embeddings: torch.Size([2, 4, 768])
Tokens: torch.Size([2, 4])
pos_embed: torch.Size([2, 4, 768])
Residual: torch.Size([2, 4, 768])
Normalized: torch.Size([2, 4, 768])
Normalized_resid_pre: torch.Size([2, 4, 768])
Residual: torch.Size([2, 4, 768])
Normalized: torch.Size([2, 4, 768])
Normalized_resid_mid: torch.Size([2, 4, 768])
Residual: torch.Size([2, 4, 768])
Normalized: torch.Size([2, 4, 768])
Normalized_resid_pre: torch.Size([2, 4, 768])
Residual: torch.Size([2, 4, 768])
Normalized: torch.Size([2, 4, 768])
Normalized_resid_mid: torch.Size([2, 4, 768])
Residual: torch.Size([2, 4, 768])
Normalized: torch.Size([2, 4, 768])
Normalized_resid_pre: torch.Size([2, 4, 768])
Residual: torch.Size([2, 4, 768])
Normalized: torch.Size([2, 4, 768])
Normalized_resid_mid: torch.Size([2, 4, 768])
Residual: torch.Size([2, 4, 768])
Normalized: torch.Size([2, 4, 768])
Normalized_resid_pre: torch.Size([2, 4, 768])
Residual: torch.Size([2, 4, 768

tensor([[[ -43.4317,  -39.8364,  -43.0660,  ...,  -54.0878,  -54.3452,
           -42.3645],
         [-128.0392, -127.9936, -130.7010,  ..., -136.7121, -129.9261,
          -129.3965],
         [-119.8521, -121.0064, -123.8820,  ..., -128.5181, -126.6027,
          -121.9060],
         ...,
         [-112.9815, -112.7749, -117.0633,  ..., -121.2914, -117.6574,
          -114.5005],
         [ -98.6724, -104.4888, -108.7361,  ..., -118.3552, -113.8766,
          -106.3604],
         [-126.8285, -128.9596, -128.3941,  ..., -140.1970, -138.5883,
          -122.3697]]], device='cuda:0', grad_fn=<AddBackward0>)

#Essayer!

In [ ]:
# Instancier le modèle DemoTransformer avec debug=False
demo_gpt2 = DemoTransformer(Config(debug=False))
# Charger le dictionnaire d'état du modèle GPT-2 de référence sans strict
demo_gpt2.load_state_dict(reference_gpt2.state_dict(), strict=False)
# Déplacer le modèle sur GPU
demo_gpt2.cuda()


DemoTransformer(
  (embed): Embed()
  (pos_embed): PosEmbed()
  (blocks): ModuleList(
    (0): TransformerBlock(
      (ln1): LayerNorm()
      (attn): Attention()
      (ln2): LayerNorm()
      (mlp): MLP()
    )
    (1): TransformerBlock(
      (ln1): LayerNorm()
      (attn): Attention()
      (ln2): LayerNorm()
      (mlp): MLP()
    )
    (2): TransformerBlock(
      (ln1): LayerNorm()
      (attn): Attention()
      (ln2): LayerNorm()
      (mlp): MLP()
    )
    (3): TransformerBlock(
      (ln1): LayerNorm()
      (attn): Attention()
      (ln2): LayerNorm()
      (mlp): MLP()
    )
    (4): TransformerBlock(
      (ln1): LayerNorm()
      (attn): Attention()
      (ln2): LayerNorm()
      (mlp): MLP()
    )
    (5): TransformerBlock(
      (ln1): LayerNorm()
      (attn): Attention()
      (ln2): LayerNorm()
      (mlp): MLP()
    )
    (6): TransformerBlock(
      (ln1): LayerNorm()
      (attn): Attention()
      (ln2): LayerNorm()
      (mlp): MLP()
    )
    (7): Transform


Prenons une chaîne de test - le paragraphe d'introduction de l'article vedette de Wikipédia aujourd'hui. Calculons la perte !

In [ ]:
test_string = """Mini scule is a species of microhylid frog endemic to Madagascar that was described in 2019. The scientific name of the species refers to its size, being a pun on the word minuscule. It is very small, measuring only 8.4 to 10.8 mm (0.33 to 0.43 in) in snout–vent length. It has bronze underparts with a brown groin and back of the thigh, cream upperparts with brown flecking, a dark brown side of the head, and a red iris. On the hind feet, the first toe is absent and the second and fifth toes are strongly reduced. The frog is known only from the Sainte Luce Reserve, where it inhabits areas with deep leaf litter near semi-permanent water bodies. Specimens of frogs from Mandena, the Vohimena mountains, the southern Anosy Mountains, and Tsitongambarika may also be of this species. Along with Mini mum and Mini ature, the other two species in its genus, it received media attention when first described due to the wordplay in its scientific name. (Full article...)"""

In [ ]:
# Convertir la chaîne de test en tokens et les déplacer sur GPU
test_tokens = reference_gpt2.to_tokens(test_string).cuda()
# Obtenir les logits du modèle DemoTransformer pour les tokens de test
demo_logits = demo_gpt2(test_tokens)


In [ ]:
def lm_cross_entropy_loss(logits, tokens):
    # Mesurer la perte du token suivant
    # Les logits ont une forme [batch, position, d_vocab]
    # Les tokens ont une forme [batch, position]
    log_probs = logits.log_softmax(dim=-1)
    pred_log_probs = log_probs[:, :-1].gather(dim=-1, index=tokens[:, 1:].unsqueeze(-1)).squeeze(-1)
    return -pred_log_probs.mean()

# Calculer la perte de la cross-entropy pour les logits de démonstration et les tokens de test
loss = lm_cross_entropy_loss(demo_logits, test_tokens)
print(loss)
print("Loss as average prob ", (-loss).exp())
print("Loss as 'uniform over this many variables'", (loss).exp())
print("Uniform loss over the vocab", math.log(demo_gpt2.cfg.d_vocab))


tensor(3.7186, device='cuda:0', grad_fn=<NegBackward0>)
Loss as average prob tensor(0.0243, device='cuda:0', grad_fn=<ExpBackward0>)
Loss as 'uniform over this many variables' tensor(41.2079, device='cuda:0', grad_fn=<ExpBackward0>)
Uniform loss over the vocab 10.82490511970208


Nous pouvons également générer du texte de manière cupide :








In [ ]:
# Initialisation de la chaîne de texte avec une actualité
test_string = "Breaking News: President Trump has been impeached by the House of Representatives for abuse of power and obstruction of Congress. The vote was 230 to 197, with 10 Republicans joining all Democrats in voting to impeach. The president is now only the third in American history to be impeached, and the first to be impeached twice. The House will now send the articles of impeachment to the Senate, where a trial will be held to determine whether to remove the president from office. The Senate is expected to begin the trial on"

# Boucle pour générer la suite du texte
for i in tqdm.tqdm(range(100)):
    # Conversion de la chaîne en tokens et envoi sur GPU
    test_tokens = reference_gpt2.to_tokens(test_string).cuda()
    # Génération de logits pour la suite du texte
    demo_logits = demo_gpt2(test_tokens)
    # Décodage et ajout du token prédit à la chaîne de texte
    test_string += reference_gpt2.tokenizer.decode(demo_logits[-1, -1].argmax())

# Impression du texte final généré
print(test_string)


  0%|          | 0/100 [00:00<?, ?it/s]

Breaking News: President Trump has been impeached by the House of Representatives for abuse of power and obstruction of Congress. The vote was 230 to 197, with 10 Republicans joining all Democrats in voting to impeach. The president is now only the third in American history to be impeached, and the first to be impeached twice. The House will now send the articles of impeachment to the Senate, where a trial will be held to determine whether to remove the president from office. The Senate is expected to begin the trial on Monday.


The House of Representatives is expected to vote on the impeachment of President Trump on Tuesday.


The House of Representatives is expected to vote on the impeachment of President Trump on Tuesday.


The Senate is expected to begin the trial on Monday.


The House of Representatives is expected to vote on the impeachment of President Trump on Tuesday.


The Senate is expected to begin the trial on Monday.


The House of Representatives is expected to vote on

# Entraînement d'un modèle !

Il s'agit d'une démonstration légère de la façon dont vous pouvez réellement entraîner votre propre GPT-2 avec ce code ! Ici, nous entraînons un petit modèle sur un petit ensemble de données, mais c'est fondamentalement le même code pour entraîner un modèle plus grand ou plus réel (bien que vous ayez besoin de GPU plus puissants et de parallélisme de données pour le faire de manière relativement efficace, et d'un parallélisme plus sophistiqué pour des modèles beaucoup plus grands)..

Pour nos besoins, nous allons entraîner un modèle de 2 couches avec 4 têtes par couche, avec une longueur de contexte de 256, pendant 1000 étapes avec une taille de lot de 32, juste pour montrer à quoi cela ressemble (et pour que le notebook ne fasse pas fondre votre colab lol).

In [ ]:
# Vérification si l'environnement est Google Colab
if IN_COLAB:
    # Installation des bibliothèques datasets et transformers
    %pip install datasets
    %pip install transformers

# Import des bibliothèques nécessaires
import datasets
import transformers
import plotly.express as px


## Config

In [ ]:
# Définition des paramètres d'entraînement
batch_size = 32  # Taille du lot
num_epochs = 10  # Nombre d'époques
max_steps = 1000  # Nombre maximum d'étapes
log_every = 10  # Fréquence de journalisation
lr = 1e-3  # Taux d'apprentissage
weight_decay = 1e-2  # Facteur de régularisation

# Configuration du modèle
model_cfg = Config(
    debug=False,            # Mode de débogage
    d_model=256,            # Dimension du modèle
    n_heads=4,              # Nombre de têtes d'attention
    d_head=64,              # Dimension des têtes d'attention
    d_mlp=1024,             # Dimension de la couche cachée
    n_layers=2,             # Nombre de couches
    n_ctx=256,              # Contexte maximum
    d_vocab=reference_gpt2.cfg.d_vocab  # Dimension du vocabulaire
)



## Créer des données

Nous chargeons un petit ensemble de données que j'ai créé, avec les 10 000 premières entrées dans le Pile (inspiré par la version de Stas pour OpenWebText !)

In [ ]:
# Chargement du jeu de données à partir du dataset Pile-10k
dataset = datasets.load_dataset("NeelNanda/pile-10k", split="train")
print(dataset)  # Affichage du dataset
print(dataset[0]['text'][:100])  # Affichage des premiers 100 caractères du texte du premier exemple

# Tokenisation et concaténation des données
tokens_dataset = tokenize_and_concatenate(dataset, reference_gpt2.tokenizer, streaming=False, max_length=model_cfg.n_ctx, column_name="text", add_bos_token=True, num_proc=4)

# Création d'un DataLoader pour charger les données en batches
data_loader = torch.utils.data.DataLoader(tokens_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)


Using custom data configuration NeelNanda--pile-10k-30e9c14cdcda6c5c
Found cached dataset parquet (/workspace/cache/NeelNanda___parquet/NeelNanda--pile-10k-30e9c14cdcda6c5c/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


Dataset({
    features: ['text', 'meta'],
    num_rows: 10000
})
It is done, and submitted. You can play “Survival of the Tastiest” on Android, and on the web. Playi
     

#0:   0%|          | 0/3 [00:00<?, ?ba/s]

#1:   0%|          | 0/3 [00:00<?, ?ba/s]

#2:   0%|          | 0/3 [00:00<?, ?ba/s]

#3:   0%|          | 0/3 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (80023 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (101051 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (155995 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (229134 > 1024). Running this sequence through the model will result in indexing errors


## Créer le Model


In [ ]:
# Création d'une instance du modèle DemoTransformer avec la configuration spécifiée
model = DemoTransformer(model_cfg)
# Transfert du modèle sur le GPU pour accélérer le traitement
model.cuda()


DemoTransformer(
  (embed): Embed()
  (pos_embed): PosEmbed()
  (blocks): ModuleList(
    (0): TransformerBlock(
      (ln1): LayerNorm()
      (attn): Attention()
      (ln2): LayerNorm()
      (mlp): MLP()
    )
    (1): TransformerBlock(
      (ln1): LayerNorm()
      (attn): Attention()
      (ln2): LayerNorm()
      (mlp): MLP()
    )
  )
  (ln_final): LayerNorm()
  (unembed): Unembed()
)

## Creer l'Optimizer
Nous utilisons AdamW - c'est un optimiseur assez standard.








In [ ]:
# Définition de l'optimiseur AdamW pour entraîner le modèle
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)


## Exécuter la boucle d'entraînement.


In [ ]:
# Liste pour stocker les valeurs de perte
# Liste pour stocker les valeurs de perte
losses = []

# Affichage du nombre de lots de données
print("Nombre de lots de données:", len(data_loader))

# Boucle sur le nombre d'époques
for epoch in range(num_epochs):
    # Boucle sur les lots de données
    for c, batch in tqdm.tqdm(enumerate(data_loader)):
        tokens = batch['tokens'].cuda()  # Transférer les tokens sur GPU
        logits = model(tokens)  # Obtenir les prédictions du modèle
        loss = lm_cross_entropy_loss(logits, tokens)  # Calcul de la perte
        loss.backward()  # Calcul du gradient
        optimizer.step()  # Mise à jour des poids du modèle
        optimizer.zero_grad()  # Réinitialisation du gradient
        losses.append(loss.item())  # Ajout de la perte à la liste
        # Affichage de la perte à intervalles réguliers
        if c % log_every == 0:
            print(f"Étape: {c}, Perte: {loss.item():.4f}")
        # Sortir de la boucle si le nombre maximum d'étapes est atteint
        if c > max_steps:
            break



Number of batches: 8506


0it [00:00, ?it/s]

Step: 0, Loss: 10.8938
Step: 10, Loss: 8.6160
Step: 20, Loss: 7.4170
Step: 30, Loss: 7.4342
Step: 40, Loss: 7.1660
Step: 50, Loss: 5.9145
Step: 60, Loss: 7.3014
Step: 70, Loss: 7.7666
Step: 80, Loss: 7.5511
Step: 90, Loss: 6.2643
Step: 100, Loss: 7.1839
Step: 110, Loss: 5.3938
Step: 120, Loss: 7.3348
Step: 130, Loss: 5.9235
Step: 140, Loss: 6.0241
Step: 150, Loss: 6.5177
Step: 160, Loss: 7.0355
Step: 170, Loss: 6.0351
Step: 180, Loss: 6.7511
Step: 190, Loss: 5.9056
Step: 200, Loss: 6.2145
Step: 210, Loss: 5.7370
Step: 220, Loss: 6.9411
Step: 230, Loss: 6.4263
Step: 240, Loss: 6.4935
Step: 250, Loss: 6.3181
Step: 260, Loss: 6.4933
Step: 270, Loss: 5.8911
Step: 280, Loss: 6.8380
Step: 290, Loss: 7.1255
Step: 300, Loss: 6.5911
Step: 310, Loss: 5.8859
Step: 320, Loss: 6.9126
Step: 330, Loss: 6.5967
Step: 340, Loss: 6.7064
Step: 350, Loss: 6.1962
Step: 360, Loss: 5.7562
Step: 370, Loss: 5.9538
Step: 380, Loss: 6.9621
Step: 390, Loss: 7.1926
Step: 400, Loss: 6.2406
Step: 410, Loss: 6.8793
St

Maintenant, nous pouvons tracer une courbe de loss curve !

In [ ]:
# Crée un graphique de ligne avec la perte en fonction du nombre de jetons traités
px.line(y=losses, x=np.arange(len(losses))*(model_cfg.n_ctx * batch_size), labels={"y":"Perte", "x":"Jetons"}, title="Courbe d'entraînement pour mon petit modèle de démonstration !")


La courbe montre que l'erreur du modèle diminue au fur et à mesure qu'il est entraîné sur plus de données. Cela signifie que le modèle apprend à identifier des modèles dans les données et à faire des prédictions plus précises.

La courbe montre également que l'erreur se stabilise après un certain nombre d'étapes d'entraînement. Cela signifie que le modèle ne s'améliore plus significativement avec un entraînement supplémentaire. C'est un phénomène courant en apprentissage automatique, et cela indique souvent que le modèle a tiré le maximum de leçons possible des données d'entraînement.